# DATA MINING FOR MOVIE RECOMMENDER SYSTEM USING CONTENT BASED FILTERING

IMPORTING LIBRARIES

In [7]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from gensim.models.doc2vec import TaggedDocument

DATA EXPLORATORY

In [8]:
#tahap awal adalah meload dataset yang digunakan
movies_data = pd.read_csv('./dataset/movies.csv')
links_data = pd.read_csv('./dataset/links.csv')
ratings_data = pd.read_csv('./dataset/ratings.csv')
tags_data = pd.read_csv('./dataset/tags.csv')


1.Dataset Movies

In [9]:
#sampel movies.csv
movies_data.sample(10)


,movieId,title,genres
5033,7835,Song of the Thin Man (1947),Comedy|Crime|Drama|Musical|Mystery|Romance
4424,6536,Sinbad: Legend of the Seven Seas (2003),Adventure|Animation|Children|Fantasy
4018,5679,"Ring, The (2002)",Horror|Mystery|Thriller
136,163,Desperado (1995),Action|Romance|Western
9739,193585,Flint (2017),Drama
8068,99149,"Misérables, Les (2012)",Drama|Musical|Romance|IMAX
7931,95583,Savages (2012),Crime|Drama|Thriller
7092,70301,Obsessed (2009),Crime|Drama|Thriller
4170,6005,Blue Collar Comedy Tour: The Movie (2003),Comedy|Documentary
1745,2340,Meet Joe Black (1998),Romance


In [10]:
#fungsi ini untuk mencheck jumlah baris dan kolom pada dataset
movies_data.shape


(9742, 3)

In [12]:
#Kemudian akan dilakukan pengecekan data type setiap kolom
movies_data.dtypes

movieId     int64
title      object
genres     object
dtype: object

In [13]:
#kita akan melakukan pengecekan apakah ada missing values pada dataset movies.csv
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [15]:
#Selanjutnya kita akan melakukan pengecekan apakah ada data ganda (duplicated)
movies_data.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
9737    False
9738    False
9739    False
9740    False
9741    False
Length: 9742, dtype: bool

2.Ratings Dataset

In [14]:
#sampel ratings.csv
ratings_data.sample(10)


,userId,movieId,rating,timestamp
65381,419,5152,2.0,1332622116
12561,80,3147,4.0,1377308769
68711,448,380,3.0,1019124334
77194,482,8528,4.0,1105396475
75066,474,34072,3.5,1141933692
57510,380,6615,4.0,1494071605
69219,448,3614,4.0,1019128089
2248,18,141688,4.0,1478124485
40831,275,3246,5.0,1049078671
42709,288,3039,3.0,975692577


In [16]:
#Mencheck jumlah baris dan kolom dataset Ratings
ratings_data.shape

(100836, 4)

In [17]:
#Mencheck tipe data setiap kolom
ratings_data.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [18]:
#Mencheck apakah terdapat missing values
ratings_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [19]:
#Mencheck apakah terdapat data ganda (duplicated)
ratings_data.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
100831    False
100832    False
100833    False
100834    False
100835    False
Length: 100836, dtype: bool

3.Dataset links

In [15]:
#Sampel links.csv
links_data.sample(10)

,movieId,imdbId,tmdbId
9019,140525,1741273,290751.0
2460,3273,134084,4234.0
5594,26900,113556,51038.0
8392,109941,2268617,83201.0
2008,2672,139809,1090.0
1868,2482,149151,1618.0
2771,3709,105428,11428.0
4158,5988,91814,21332.0
1737,2332,158493,12888.0
1395,1913,73540,11020.0


In [21]:
#Mencheck jumlah baris dan kolom dataset links
links_data.shape

(9742, 3)

In [22]:
#Mencheck tipe data setiap kolom
links_data.dtypes

movieId      int64
imdbId       int64
tmdbId     float64
dtype: object

In [23]:
#Mencheck apakah terdapat missing values
links_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9742 non-null   int64  
 1   imdbId   9742 non-null   int64  
 2   tmdbId   9734 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 228.5 KB


In [24]:
#Selanjutnya kita akan melakukan pengecekan apakah ada data ganda (duplicated)
links_data.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
9737    False
9738    False
9739    False
9740    False
9741    False
Length: 9742, dtype: bool

4.Dataset Tags

In [16]:
#Sampel tags.csv
tags_data.sample(10)

,userId,movieId,tag,timestamp
707,424,147,Mark Wahlberg,1457901624
410,103,296,great soundtrack,1431954555
1665,474,2699,spiders,1137373650
355,62,135536,visually appealing,1525555109
3282,567,161634,intense,1525287241
3529,599,296,sarcasm,1498456612
2357,474,8580,In Netflix queue,1137201021
3628,599,2959,challenging,1498456963
508,184,4226,Mindfuck,1537094464
362,62,136864,Gal Gadot,1525555240


In [25]:
#Mencheck jumlah baris dan kolom dataset tags
tags_data.shape

(3683, 4)

In [26]:
#Mencheck tipe data setiap kolom
tags_data.dtypes

userId        int64
movieId       int64
tag          object
timestamp     int64
dtype: object

In [27]:
#Mencheck apakah terdapat missing values
tags_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3683 entries, 0 to 3682
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   userId     3683 non-null   int64 
 1   movieId    3683 non-null   int64 
 2   tag        3683 non-null   object
 3   timestamp  3683 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 115.2+ KB


In [28]:
#Selanjutnya kita akan melakukan pengecekan apakah ada data ganda (duplicated)
links_data.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
9737    False
9738    False
9739    False
9740    False
9741    False
Length: 9742, dtype: bool